# Song Classification with the GTZAN Dataset

In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision
import imageio.v3
import librosa
import librosa.display

from tqdm import trange


In [2]:
genres = os.listdir('Data/genres_original/')
print(genres)
genre_dict = {genres[i]: i for i in range(len(genres))}
genre_dict

['pop', 'metal', 'disco', 'blues', 'reggae', 'classical', 'rock', 'hiphop', 'country', 'jazz']


{'pop': 0,
 'metal': 1,
 'disco': 2,
 'blues': 3,
 'reggae': 4,
 'classical': 5,
 'rock': 6,
 'hiphop': 7,
 'country': 8,
 'jazz': 9}

In [3]:
three_sec_df = pd.read_csv('Data/features_3_sec.csv')

# changing dots in middle of file name to _ because it makes more sense
three_sec_df['filename'] = three_sec_df['filename'].apply(lambda x: f'{x[:-4].replace(".", "_")}.wav')

three_sec_df

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues_00000_0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues_00000_1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues_00000_2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues_00000_3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues_00000_4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,rock_00099_5.wav,66149,0.349126,0.080515,0.050019,0.000097,1499.083005,164266.886443,1718.707215,85931.574523,...,42.485981,-9.094270,38.326839,-4.246976,31.049839,-5.625813,48.804092,1.818823,38.966969,rock
9986,rock_00099_6.wav,66149,0.372564,0.082626,0.057897,0.000088,1847.965128,281054.935973,1906.468492,99727.037054,...,32.415203,-12.375726,66.418587,-3.081278,54.414265,-11.960546,63.452255,0.428857,18.697033,rock
9987,rock_00099_7.wav,66149,0.347481,0.089019,0.052403,0.000701,1346.157659,662956.246325,1561.859087,138762.841945,...,78.228149,-2.524483,21.778994,4.809936,25.980829,1.775686,48.582378,-0.299545,41.586990,rock
9988,rock_00099_8.wav,66149,0.387527,0.084815,0.066430,0.000320,2084.515327,203891.039161,2018.366254,22860.992562,...,28.323744,-5.363541,17.209942,6.462601,21.442928,2.354765,24.843613,0.675824,12.787750,rock


In [4]:
thirty_sec_df = pd.read_csv('Data/features_30_sec.csv')
thirty_sec_df['filename'] = thirty_sec_df['filename'].apply(lambda x: f'{x[:-4].replace(".", "_")}.wav')
thirty_sec_df = thirty_sec_df.drop([554]) # dropping entry that does not have spectrogram
thirty_sec_df


,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues_00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues_00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues_00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues_00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues_00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,rock_00095.wav,661794,0.352063,0.080487,0.079486,0.000345,2008.149458,282174.689224,2106.541053,88609.749506,...,45.050526,-13.289984,41.754955,2.484145,36.778877,-6.713265,54.866825,-1.193787,49.950665,rock
996,rock_00096.wav,661794,0.398687,0.075086,0.076458,0.000588,2006.843354,182114.709510,2068.942009,82426.016726,...,33.851742,-10.848309,39.395096,1.881229,32.010040,-7.461491,39.196327,-2.795338,31.773624,rock
997,rock_00097.wav,661794,0.432142,0.075268,0.081651,0.000322,2077.526598,231657.968040,1927.293153,74717.124394,...,33.597008,-12.845291,36.367264,3.440978,36.001110,-12.588070,42.502201,-2.106337,29.865515,rock
998,rock_00098.wav,661794,0.362485,0.091506,0.083860,0.001211,1398.699344,240318.731073,1818.450280,109090.207161,...,46.324894,-4.416050,43.583942,1.556207,34.331261,-5.041897,47.227180,-3.590644,41.299088,rock


In [5]:
thirty_sec_df.loc[thirty_sec_df['filename'] == 'jazz_00054.wav']

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label


In [6]:
# Removing the "." from the middle of file names and replacing with "_" in audio folders to match the spectrogram files and also because it's really stupid to put a . in the middle of a filename
# Run this only once because otherwise it won't work if you try it again, but probably won't break anything i guess
for genre in genres:
    for song_id in os.listdir(f'Data/genres_original/{genre}'):
        #print(song_id[:-4].replace(".", ""))
        os.rename(f'Data/genres_original/{genre}/{song_id}', f'Data/genres_original/{genre}/{song_id[:-4].replace(".", "_")}.wav')

In [7]:
class AudioDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        genre_number = genre_dict[self.data.iloc[idx]['label']]
        y = [1 if i == genre_number  else 0 for i in range(len(genres))]
        x, _ = librosa.load(f'Data/genres_original/{genres[genre_number]}/{self.data.iloc[idx]["filename"][:-4]}.wav')
        #print(x.shape)
        if x.shape[0] < 675808:
            x = np.append(x, np.zeros(675808 - len(x)))
            #print(type(x[0]))
         
        return torch.Tensor(np.array([x.astype('float32')])), torch.Tensor(np.array(y).astype('float32'))
        

In [8]:
thirty_sec_df.iloc[0]['label']

'blues'

In [9]:
batch_size = 128

class SpectrogramDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        genre_number = genre_dict[self.data.iloc[idx]['label']]
        #print(self.data.iloc[idx]['label'])
        y = [1 if i == genre_number  else 0 for i in range(len(genres))]
        #print(genre_number)
        #print(y)
        #print(genres[y])
        image = imageio.v3.imread(f'Data/images_original/{genres[genre_number]}/{self.data.iloc[idx]["filename"][:-4].replace("_", "")}.png')
        x = torch.Tensor(image)
        x = torch.reshape(x, (4, 288, 432))
        #print(x.shape)
       
        #plt.imshow(x)
        return x, torch.Tensor(y)


In [25]:


train_set, test_set = train_test_split(thirty_sec_df, test_size = 0.2)
train_set, val_set = train_test_split(train_set, test_size = 0.2)

audio_train = AudioDataset(train_set)
audio_val = AudioDataset(val_set)
audio_test = AudioDataset(test_set)

audio_train_dl = DataLoader(audio_train, batch_size=1)
audio_val_dl = DataLoader(audio_val, batch_size=1)
audio_test_dl = DataLoader(audio_test, batch_size=1)

spec_train = SpectrogramDataset(train_set)
spec_val = SpectrogramDataset(val_set)
spec_test = SpectrogramDataset(test_set)

spec_train_dl = DataLoader(spec_train, shuffle=True, batch_size=batch_size)
spec_val_dl = DataLoader(spec_val, shuffle=True, batch_size=batch_size)
spec_test_dl = DataLoader(spec_test, shuffle=True, batch_size=batch_size)



In [26]:
for i in range(50):
    print(audio_train[i][0][0])



tensor([-0.0422, -0.1986, -0.2465,  ...,  0.0000,  0.0000,  0.0000])
tensor([ 0.0286,  0.0132, -0.0219,  ...,  0.0000,  0.0000,  0.0000])
tensor([0.1663, 0.1911, 0.1969,  ..., 0.0000, 0.0000, 0.0000])
tensor([0.0563, 0.0918, 0.1017,  ..., 0.0000, 0.0000, 0.0000])
tensor([0.0624, 0.0481, 0.0356,  ..., 0.0000, 0.0000, 0.0000])
tensor([-0.0177,  0.0165,  0.0522,  ...,  0.0000,  0.0000,  0.0000])
tensor([ 0.0032, -0.0070, -0.0274,  ...,  0.0000,  0.0000,  0.0000])
tensor([-0.0294, -0.0371, -0.0275,  ...,  0.0000,  0.0000,  0.0000])
tensor([0.0208, 0.0875, 0.0679,  ..., 0.0000, 0.0000, 0.0000])
tensor([0.0401, 0.0560, 0.0395,  ..., 0.0000, 0.0000, 0.0000])
tensor([ 0.0050, -0.0015, -0.0091,  ...,  0.0000,  0.0000,  0.0000])
tensor([-0.0373,  0.0169,  0.0081,  ...,  0.0000,  0.0000,  0.0000])
tensor([-0.0721, -0.0685, -0.0671,  ...,  0.0000,  0.0000,  0.0000])
tensor([-0.0304, -0.0220, -0.0236,  ...,  0.0000,  0.0000,  0.0000])
tensor([-0.0648, -0.1072, -0.1094,  ...,  0.0000,  0.0000,  0.00

In [12]:
for i in range(50):
    print(spec_train[i][0].shape)

/var/folders/5k/k9z7r_vs0098fm1rwf519ml40000gn/T/ipykernel_98697/4291303028.py:18: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525473998/work/torch/csrc/utils/tensor_numpy.cpp:205.)
  x = torch.Tensor(image)


torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([4, 288, 432])
torch.Size([

that spectrogram looks awful but that's just some torch tensor garbage, the actual spectrograms are still good
matplotlib just doesn't like torch tensors

In [13]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


input_shape = (batch_size, 4, 288, 432)

class FeedforwardSpectrogramModel(nn.Module):

    def __init__(self):
        
        super().__init__()
        
        self.conv1 = nn.Conv2d(4, 32, 3, stride=2)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout()
        self.norm1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.norm2 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, stride=2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout()
        self.norm3 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.norm4 = nn.BatchNorm2d(64)
        self.dropout3 = nn.Dropout()
        
        self.norm5 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, stride=2)
        self.relu3 = nn.ReLU()
        self.dropout4 = nn.Dropout()
        self.norm6 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.norm7 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 1, stride=2)
        self.relu4 = nn.ReLU()
        self.dropout5 = nn.Dropout()
        self.norm8 = nn.BatchNorm2d(256)
        self.pool4 = nn.MaxPool2d(2, 2)
        self.norm9 = nn.BatchNorm2d(256)
        self.dropout6 = nn.Dropout()
        
        
        self.conv5 = nn.Conv2d(32, 64, 3, stride=2)
        self.relu5 = nn.ReLU()
        self.pool5 = nn.MaxPool2d(2, 2)
        self.conv6 = nn.Conv2d(64, 64, 1, stride=2)
        self.relu6 = nn.ReLU()
        self.dropout3 = nn.Dropout()

        self.flatten = nn.Flatten()
        
        self.dense = nn.Linear(256, 10)
        self.softmax = nn.LogSoftmax()
        
    def forward(self, x):
        #print('before first conv block')
        
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.norm1(x)
        x = self.pool1(x)
        x = self.norm2(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.dropout5(x)
        x = self.norm3(x)
        x = self.pool2(x)
        x = self.norm4(x)
        x = self.dropout6(x)

        #print('before second conv block')
        x = self.norm5(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.dropout4(x)
        x = self.norm6(x)
        x = self.pool3(x)
        x = self.norm7(x)
        x = self.conv4(x)
        x = self.dropout5(x)
        x = self.relu4(x)
        x = self.pool4(x)
        x = self.norm8(x)
        x = self.dropout6(x)
     
        

        """
        #print('before third conv block')
        x = self.conv5(x)
        x = self.pool5(x)
        print('before conv6')
        x = self.conv6(x)
        x = self.pol(6)
        x = self.dropout3(x)
        x = self.relu3(x)
        """
        
        x = self.flatten(x)
        x = self.dense(x)
        out = self.softmax(x)

        return out

model = FeedforwardSpectrogramModel().to(DEVICE)
print(model)

FeedforwardSpectrogramModel(
  (conv1): Conv2d(4, 32, kernel_size=(3, 3), stride=(2, 2))
  (relu1): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (norm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
  (relu2): ReLU()
  (dropout2): Dropout(p=0.5, inplace=False)
  (norm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (norm4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout3): Dropout(p=0.5, inplace=False)
  (norm5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(

In [90]:
def train_one_epoch(train_loader, model, device, optimizer, log_interval, epoch):
    model.train()
    losses = []
    counter = []

    for i, (img, label) in enumerate(train_loader):
        #print(img)
        #print(img.shape)
        #print(label)

        #print(label.shape)
        #print(img.shape)

        #print(img)
        
        img = img.to(device)
        label = label.to(device)
        # img = torch.flatten(img)
        #print((img.dtype))
        #print(img.shape)
        #print(img)
    
        optimizer.zero_grad()

        
        output = model(img)

        #print(output.shape)
        output = torch.reshape(output, (1, 10))

        #print(output.shape)
        #print(label.shape)

        criterion = nn.CrossEntropyLoss()

        #print(type(output.data.dtype))
        #print(type(label.data.dtype))

        
        

        pred = torch.Tensor([output.data.max().item()]).type(torch.LongTensor)
        label_idx = torch.Tensor([label.data.max().item()]).type(torch.LongTensor)
        #print(output)
        #print(label)
        #print(pred)
        #print(label_idx)
        #print(pred.shape)
        #print(label_idx.shape)
        
        loss = criterion(output, label)
        loss.backward()

        optimizer.step()

        if (i+1) % log_interval == 0:
            losses.append(loss.item())
            counter.append(
                (i * batch_size) + img.size(0) + epoch * len(train_loader.dataset))
    
    return losses, counter

def test_one_epoch(test_loader, model, device):
    model.eval()
    test_loss = 0
    num_correct = 0
    
    with torch.no_grad():
        for i, (img, label) in enumerate(test_loader):
            img, label = img.to(device), label.to(device)

            # ------------------
            # Write your implementation here.
            
            output = model(img)
            output = torch.reshape(output, (1, 10))
            pred = output.data.max(1, keepdim=True)[1] # Get index of largest log-probability and use that as prediction
            
            #print(pred)
            
            label_idx = label.data.max(1, keepdim=True)[1]
            #print(pred.shape)
            #print(label_idx.shape)

            #print(label)
            num_correct += output.eq(label.data.view_as(output)).sum()
            criterion = nn.CrossEntropyLoss()
            #print(output)
            #print(label)
            #print(output.shape)
            #print(label.shape)
            test_loss += criterion(output, label)

            # ------------------
            
    #test_loss /= len(test_loader.dataset)
    return test_loss / (len(test_loader.dataset) / batch_size), num_correct

In [89]:
train_loader = spec_val_dl
val_loader = spec_val_dl


# Hyperparameters
lrs = [0.0001, 0.0003, 0.0005, 0.0007, 0.001, 0.003, 0.005, 0.007, 0.085, 0.01, 0.03]
max_epochs = 5
gamma = 0.95

# Recording data
log_interval = len(train_loader) / batch_size
def tune_model(lrs, n_epochs, train_loader=train_loader, val_loader=val_loader, gamma=gamma, type='spectrogram', num_channels=1, dilation_depth=2, num_repeat=1):
    if type == 'wavenet':
        model = WaveNet(num_channels, dilation_depth, num_repeat).to(DEVICE)
    else:
        model = FeedforwardSpectrogramModel().to(DEVICE)

        model = model.float()
        
    lr_dict = {}
    for lr in lrs:
        # Instantiate optimizer (model was created in previous cell)
        optimizer = torch.optim.Adagrad(model.parameters(), lr=lr)

        train_losses = []
        train_counter = []
        val_losses = []
        val_correct = []
        for epoch in trange(n_epochs, leave=True, desc='Epochs'):
            train_loss, counter = train_one_epoch(train_loader, model, DEVICE, optimizer, log_interval, epoch)
            val_loss, num_correct = test_one_epoch(val_loader, model, DEVICE)

            # Record results
            train_losses.extend(train_loss)
            train_counter.extend(counter)
            val_losses.append(val_loss)
            val_correct.append(num_correct)

        print(f"Val accuracy: {val_correct[-1]/len(val_loader.dataset)}")
        lr_dict[lr] = train_losses, train_counter, val_losses, val_correct

results = tune_model(lrs, max_epochs, type='wavenet', train_loader=audio_val_dl, val_loader=audio_val_dl, num_channels=1, num_repeat=2)


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]/var/folders/5k/k9z7r_vs0098fm1rwf519ml40000gn/T/ipykernel_98697/2132776828.py:96: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.softmax(out)


tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])
torch.Size([1, 10])
torch.Size([1, 10])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])
torch.Size([1, 10])
torch.Size([1, 10])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])
torch.Size([1, 10])
torch.Size([1, 10])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])
torch.Size([1, 10])
torch.Size([1, 10])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])
torch.Size([1, 10])
torch.Size([1, 10])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])
torch.Size([1, 10])
torch.Size([1, 10])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])
torch.Size([1, 10])
torch.Size([1, 10])
tensor

Epochs:  20%|██        | 1/5 [01:17<05:09, 77.26s/it]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])
torch.Size([1, 10])
torch.Size([1, 10])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])
torch.Size([1, 10])
torch.Size([1, 10])


Epochs:  20%|██        | 1/5 [02:02<08:11, 122.92s/it]


KeyboardInterrupt: 

In [ ]:
print(val_losses)

NameError: name 'val_losses' is not defined

In [ ]:
print(train_losses)

NameError: name 'train_losses' is not defined

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.plot(train_counter, train_losses, label='Train loss')
plt.plot([i * len(train_loader.dataset) for i in range(1, max_epochs + 1)], 
        val_losses, label='Test loss', marker='o')
plt.xlim(left=0)
plt.ylim(bottom=0)
plt.title('Loss curve', fontsize=24)
plt.xlabel('Number of training examples seen', fontsize=16)
plt.ylabel('Cross Entropy Loss', fontsize=16)
plt.legend(loc='upper right', fontsize=14)

NameError: name 'train_counter' is not defined

<Figure size 1200x800 with 0 Axes>

In [ ]:
torch.save(model, '008lr_100epochs.model')

# Classifying the Raw Audio Files

In [66]:
class CausalConv1d(torch.nn.Conv1d):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, dilation=1, groups=1, bias=True):
        self.__padding = (kernel_size - 1) * dilation

        super(CausalConv1d, self).__init__(
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=self.__padding,
            dilation=dilation,
            groups=groups,
            bias=bias,
        )


    def forward(self, input):
        #print(input.shape)
        #print(self.in_channels)
        #print(self.out_channels)
        result = super(CausalConv1d, self).forward(input)

        if self.__padding != 0:
            return result[:, :, : -self.__padding]
        return result

def _conv_stack(dilations, in_channels, out_channels, kernel_size):
    """
    Create stack of dilated convolutional layers, outlined in WaveNet paper:
    https://arxiv.org/pdf/1609.03499.pdf
    """
    return nn.ModuleList(
        [
            CausalConv1d(
                in_channels=in_channels,
                out_channels=out_channels,
                dilation=d,
                kernel_size=kernel_size,
            )
            for i, d in enumerate(dilations)
        ]
    )

class WaveNet(nn.Module):
    def __init__(self, num_channels, dilation_depth, num_repeat, kernel_size=2):
        super(WaveNet, self).__init__()
        dilations = [2 ** d for d in range(dilation_depth)] * num_repeat
        internal_channels = int(num_channels * 2)
        self.hidden = _conv_stack(dilations, num_channels, internal_channels, kernel_size)
        self.residuals = _conv_stack(dilations, num_channels, num_channels, 1)
        self.input_layer = CausalConv1d(
            in_channels=1,
            out_channels=num_channels,
            kernel_size=1,
            groups=1,
        )

        self.linear_mix = nn.Conv1d(
            in_channels=num_channels * dilation_depth * num_repeat,
            out_channels=1,
            kernel_size=1,
        )
        self.num_channels = num_channels

        self.softmax = nn.LogSoftmax(dim=1)

        self.dense = nn.Linear(675808, 10)

    def forward(self, x):
        out = x
        skips = []

        #print(self.input_layer.__str__)
        
        out = self.input_layer(out)

        for hidden, residual in zip(self.hidden, self.residuals):
            x = out
            out_hidden = hidden(x)

            # gated activation
            #   split (32,16,3) into two (16,16,3) for tanh and sigm calculations
            out_hidden_split = torch.split(out_hidden, self.num_channels, dim=1)
            out = torch.tanh(out_hidden_split[0]) * torch.sigmoid(out_hidden_split[1])

            skips.append(out)

            out = residual(out)
            out = out + x[:, :, -out.size(2) :]

        # modified "postprocess" step:
        out = torch.cat([s[:, :, -out.size(2) :] for s in skips], dim=1)
        out = self.linear_mix(out)

        out = self.dense(out)
        out = self.softmax(out) 
        
        return out
    
wavenet = WaveNet(num_channels=4, dilation_depth=3, num_repeat=4)
        